In this notebook, we construct manually the tensor for the 1D hubbard model. The purpose of this is to demonstrate the principles of the manual construction of a MPO.

The MPO in question will make explicit use of particle number conservation in its structure.


## Building a MPO

The usual strategy when manually building a MPO is to view the rank 4 tensors that makes it up as a matrices of quantum operators, inserting elements in it such that a matrix product of many such matrices accumulates the Hamiltonian in the lower left corner.

Within this framework, a translation invariant 1D Hubbard hamiltonian 

$$
H = \sum_{i \sigma}\left( -t (c^\dagger_{i\sigma} c_{i+1,\sigma} + c^\dagger_{i+1,\sigma}c_{i\sigma}) -\mu n_{i\sigma} + \frac{U}{2} n_{i\uparrow}n_{i\downarrow} \right)
$$

can be built from the following matrix of operators:


$$
\newcommand{\Id}{\unicode{x1d7d9}}
T_i = \begin{array}{ c c c c c c}
 I & 0 & 0 & 0 & 0 & 0 \\
 \tilde{c}_{i\uparrow} & 0 & 0 & 0 & 0  & 0 \\
 \tilde{c}_{i\downarrow} & 0 & 0 & 0 & 0  & 0 \\
 \tilde{c}_{i\uparrow}^\dagger & 0 & 0 & 0 & 0  & 0 \\
 \tilde{c}_{i\downarrow}^\dagger & 0 & 0 & 0 & 0  & 0 \\
 U n_{i\uparrow} n_{i\downarrow} - \mu (n_{i\uparrow} + n_{i\downarrow}) 
 & t  F_i \tilde{c}^\dagger_{i\uparrow} & t F_i \tilde{c}_{i\downarrow}^\dagger  
 & t \tilde{c}_{i\uparrow} F_i & t \tilde{c}_{i\downarrow} F_i & I \\
\end{array}
$$

and the lower left corner of $T_i T_{i+1}$ is $ \sum_{j = i}^{i+1} \left( U n_{j\uparrow} n_{j\downarrow} - \mu (n_{j\uparrow} + n_{j\downarrow})\right) + \sum_\sigma -t \left( \tilde{c}_{i\sigma}^\dagger F_i \tilde{c}_{i+1,\sigma}  + \tilde{c}_{i+1\sigma}^\dagger F_{i} \tilde{c}_{i,\sigma}\right).$

We will now construct this tensor using Quantit's tools.

To do so we must first identify the conserved quantity to assign to each index. A conserving Hamiltonian always leaves a conserved quantity unchanged, its overall selection rule must be 0 in this case. What goes on the bonds is the inverse of the selection rule of the operator we have put there. 
The row must then have [0,1,1,-1,-1,0] and the columns [0,-1,-1,1,1,0]

In [14]:
import quantit as qtt


pnum = qtt.conserved.Z;
U = qtt.full([],pnum(0),6)
mu = qtt.full([],pnum(0),3)
t = qtt.full([],pnum(0),1)

HS = qtt.btensor([[[1,pnum(0)],[2,pnum(1)],[1,pnum(2)]]],pnum(0))
print(HS)


btensor rank 1
 selection rule [Z(0)]
 number of sections by dim [3]
 sections sizes [1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(2)]]



First we defined a shorthand for the conserved value type and variables for the hamiltonian parameters.
We then construct an empty btensor as a device to describe the local hilbert space of the electrons: one empty state, 2 states with one electron and one state with two electrons.

Next, we use the *fermions* function from *quantit.operators* in tandem with the Hilbert space descriptor to generate the local fermions annihilation and phase operators with our choosen conservation law.

In [16]:


c_up,c_dn,F,id = qtt.operators.fermions(qtt.shape_from([HS,HS.conj()]))
c_dag_up = c_up.conj().permute([1,0])
c_dag_dn = c_dn.conj().permute([1,0])

n_up = c_dag_up.bmm(c_up)
n_dn = c_dag_dn.bmm(c_dn)

H_l = -mu * (n_up + n_dn) + U*(n_up.bmm(n_dn))


From the annihilation operators, we constructed the creation operator by applying conjugation (of complex number and conserved quantities) and transposing using *conj* and *permute* methods. Then, we prepared the number operator by performing matrix multiplication of creation and annihilation operator. Finally, we compute the local term of the Hubbard hamiltonian.

Next, we prepare the description of the vector space for the MPO bonds and compose the bonds shape with the Hilbert space description into the shape of the tensor we will put in the MPO.

In [17]:

leftbond = qtt.btensor([[[1,pnum(0)],[2,pnum(1)],[2,pnum(-1)],[1,pnum(0)]]],pnum(0))
rightbond = leftbond.conj()

T = qtt.shape_from([leftbond,HS,rightbond,HS.conj()])

Finally, we insert the operators in the MP. We first fill in the leftmost column, then the bottom row using *basic_index_put_*.

This function takes a list of integer and a velue in the form of a *torch.tensor* or a *quantit btensor*. The value is put into a view on the whole tensor. A whole dimensiosn is kept when we give the index **-1** to that dimension. For exemple, *[0,-1,0,-1]* signify that we want to insert into the topleft element when the tensor is viewed as a matrix of operator.

After we have inserted all the elements in T, we create a *quantit.networks.bMPO* (block-MPO) of 20 sites with the same T tensor at all sites. Then, some adjustement need to made on the edge tensor to account for the finiteness of the hamiltonian: we keep only the last row of the first tensor and the first column of the last tensor. We then us the *coalesce* method to optimize the block structure of the MPO. The choice to accumulate the hamiltonian in the lower left corner is practical for us to reason and prepare the tensor, but its not the most efficient way to organize the block tensor. *coalesce* does a gauge transform such that there's no two blocks within a tensor with exactly the same conserved value combination on its dimensions.

In [18]:
T.basic_index_put_([0,-1,0,-1], id)
T.basic_index_put_([1,-1,0,-1], c_up)
T.basic_index_put_([2,-1,0,-1], c_dn)
T.basic_index_put_([3,-1,0,-1], c_dag_up)
T.basic_index_put_([4,-1,0,-1], c_dag_dn)
T.basic_index_put_([5,-1,0,-1], H_l)
T.basic_index_put_([5,-1,1,-1], t*F.bmm(c_dag_up))
T.basic_index_put_([5,-1,2,-1], t*F.bmm(c_dag_dn))
T.basic_index_put_([5,-1,3,-1], t*qtt.bmm(c_up,F))
T.basic_index_put_([5,-1,4,-1], t*qtt.bmm(c_dn,F))
T.basic_index_put_([5,-1,5,-1], id)


H = qtt.networks.bMPO(20,T)
H[0] = H[0].basic_create_view([5,-1,-1,-1],preserve_rank=True)
H[19] = H[19].basic_create_view([-1,-1,0,-1],preserve_rank=True)
H.coalesce()